# Project 2: Global alignment with different gap costs
This project is about implementing and experimenting with pairwise sequence comparison methods to compute optimal global alignments of two sequences where the object is to minimize a cost.

### Clearing up conceptual details

Pairwise alignment minimizing cost: the difference compared to maximizing cost as we did with the first project is in the scoring matrix, in case of cost minimizing, it gives a score of 0 (= not rewarding) to matches and positive score to mismatches, while with maximizing cost, we reward matches with a higher positive score and not give that high of a score to mismatches

* "Maximizing cost": the goal is to have the highest number of matches, the total length of the aligned sequences doesn't matter
* "Minimizing cost": the goal is to obtain the shortest possible alignment

## Reading in a FASTA file

In [32]:
from pysam import FastaFile

sequences_object = FastaFile("../data/case_01.fasta")

seq1 = sequences_object.fetch("seq1").upper()
seq2 = sequences_object.fetch("seq2").upper()
print(seq1)

ACGTGTCAACGT


## Specifying the score matrix

Module "score_matrix.py"

In [19]:
def initiate_score_matrix(file):
        raw = open(file, "r").read()
        lines_sep = raw.split('\n')
        nested_dict = {}

        for line in lines_sep:
            parsed_line = line.split()
            key = parsed_line[0]
            scores_per_nucl = {}

            for char in range(1, len(parsed_line)):
                # Defining the inner dictionary
                scores_per_nucl[lines_sep[char-1][0]] = int(parsed_line[char])
            
            nested_dict[key] = scores_per_nucl
        
        return nested_dict

## Problem I: Implementing alignment methods
You should implement the following programs for pairwise global alignment of DNA sequences (i.e. alphabeth a, c, g, t) based on minimizing cost, i.e. the optimal score is the minimum score.

### 1. Mandatory:
Make a program global_linear that implements global alignment using linear gap cost. The program should be implemented such that it takes at most quadratic time and space to compute the cost of an optimal global alignment. If requested, the program should output an optimal global alignment.

So basically the same as last time, only with minimizing instead of maximizing

"If requested, the program should output an optimal global alignment" = We have to have backtracking implemented as well

In [25]:
sc = initiate_score_matrix("../data/score_matrix.txt")
sc

{'A': {'A': 0, 'C': 5, 'G': 2, 'T': 5},
 'C': {'A': 5, 'C': 0, 'G': 5, 'T': 2},
 'G': {'A': 2, 'C': 5, 'G': 0, 'T': 5},
 'T': {'A': 5, 'C': 2, 'G': 5, 'T': 0}}

In [28]:
def choose_gapcost():
    while True:
        lin_or_aff = input("Linear/affine gapcost: ").lower()
        if lin_or_aff == "linear":
            user_input = input("Gap cost: ")
            if user_input.isdigit():
                gap_cost = 5 * int(user_input)
                break
            print("Invalid input. Please try again.")
        elif lin_or_aff == "affine":
            user_input = input("Gap cost: ")
            if user_input.isdigit():
                gap_cost = 5 + 5 * int(user_input)
                break
            print("Invalid input. Please try again.")
        print("Invalid input. Please try again.")
    return gap_cost

choose_gapcost()

25

In [40]:
# With Backtracking
def needleman_wunsch(seq1, seq2, matrix_file):

    score_matrix = initiate_score_matrix(matrix_file)

    # Request user input for the gap cost, having the choice for either linear/affine
    def choose_gapcost():
        while True:
            lin_or_aff = input("Linear/affine gapcost: ").lower()
            if lin_or_aff == "linear":
                gc_input = input("Gap cost: ")
                if gc_input.isdigit():
                    gap_cost = int(gc_input)
                    break
                print("Invalid input. Please try again.")
            elif lin_or_aff == "affine":
                go_input = input("Gap open: ")
                if go_input.isdigit():
                    continue
                ge_input = input("Gap extend: ")
                if ge_input.isdigit():
                    gap_cost = int(go_input) + int(ge_input)
                    break
                print("Invalid input. Please try again.")
            print("Invalid input. Please try again.")
        return gap_cost
    
    gap_cost = choose_gapcost()

    n, m = len(seq1), len(seq2)

    # Initialize dynamic programming table filled with zeros
    dp = [[0] * (m + 1) for _ in range(n + 1)]

    # Initialize the first row and column with gap penalties
    for i in range(1, n + 1):
        dp[i][0] = dp[i - 1][0] + gap_cost
    for j in range(1, m + 1):
        dp[0][j] = dp[0][j - 1] + gap_cost

    # Fill in the rest of the matrix
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            match_score = dp[i - 1][j - 1] + score_matrix[seq1[i - 1]][seq2[j - 1]]
            delete_score = dp[i - 1][j] + gap_cost
            insert_score = dp[i][j - 1] + gap_cost
            dp[i][j] = min(match_score, delete_score, insert_score)

    # Backtrack to find the alignment
    aligned_seq1, aligned_seq2 = "", ""
    i, j = n, m
    while i > 0 or j > 0:
        if i > 0 and dp[i][j] == dp[i - 1][j] + gap_cost:
            aligned_seq1 = seq1[i - 1] + aligned_seq1
            aligned_seq2 = "-" + aligned_seq2
            i -= 1
        elif j > 0 and dp[i][j] == dp[i][j - 1] + gap_cost:
            aligned_seq1 = "-" + aligned_seq1
            aligned_seq2 = seq2[j - 1] + aligned_seq2
            j -= 1
        else:
            aligned_seq1 = seq1[i - 1] + aligned_seq1
            aligned_seq2 = seq2[j - 1] + aligned_seq2
            i -= 1
            j -= 1
    return aligned_seq1, aligned_seq2, dp[n][m]

In [41]:
aligned_seq1, aligned_seq2, alignment_score = needleman_wunsch(seq1, seq2, "../data/score_matrix.txt")
print("Aligned Sequence 1:", aligned_seq1)
print("Aligned Sequence 2:", aligned_seq2)
print("Alignment Score:", alignment_score)

Aligned Sequence 1: ACGT-GTCAACGT-
Aligned Sequence 2: ACGTCGT-AGC-TA
Alignment Score: 22


### 2. Mandatory:
Make a program global_affine that implements global alignment using affine gap cost. The program should be implemented such that it takes at most quadratic time and space to compute the cost of an optimal global alignment. If requested, the program should output an optimal global alignment.